In [2]:
from nltk.corpus import wordnet

In [3]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("text_emotion.csv")
df = df.drop(["tweet_id", "author"], axis=1)
df.head(10)

sentiment                                            content
0       empty  @tiffanylue i know  i was listenin to bad habi...
1     sadness  Layin n bed with a headache  ughhhh...waitin o...
2     sadness                Funeral ceremony...gloomy friday...
3  enthusiasm               wants to hang out with friends SOON!
4     neutral  @dannycastillo We want to trade with someone w...
5       worry  Re-pinging @ghostridah14: why didn't you go to...
6     sadness  I should be sleep, but im not! thinking about ...
7       worry               Hmmm. http://www.djhero.com/ is down
8     sadness            @charviray Charlene my love. I miss you
9     sadness         @kelcouch I'm sorry  at least it's Friday?

In [7]:
import re
sentence = []
universal_word_list = []
maximum_length = 0
for i in df.content:
    local = ""
    flag = 0
    wordList = re.sub("[^\w]", " ",  i).split()
    for j in wordList:
        if not wordnet.synsets(j):
            continue
        elif "@" in j:
            flag = flag + 1
            local = local + "[USER] "
        else:
            flag = flag + 1
            j = j.lower()
            if j not in universal_word_list:
                universal_word_list.append(j)
            local = local + j + " "
    if maximum_length <= flag:
        maximum_length = flag
    sentence.append(local)

In [8]:
print(sentence[:10])
print("Maximum length of the sentence in the dataset", maximum_length)
content = pd.DataFrame({'sentences':sentence})
print(content.head(10))
df = df.drop(["content"], axis=1)
df['content'] = content.values
df.head(10)

['i know i was bad habit earlier i started at part ', 'n bed a headache on call ', 'funeral ceremony gloomy friday ', 'wants hang out friends soon ', 'want trade someone who has houston tickets but no one will ', 're pinging why t go prom bc t like friends ', 'i be sleep but not thinking about an old friend who i want but he s married now damn amp he wants me 2 scandalous ', 'http www is down ', 'love i miss ', 'i m sorry at least it s friday ']
Maximum length of the sentence in the dataset 31
                                           sentences
0  i know i was bad habit earlier i started at part 
1                          n bed a headache on call 
2                    funeral ceremony gloomy friday 
3                       wants hang out friends soon 
4  want trade someone who has houston tickets but...
5        re pinging why t go prom bc t like friends 
6  i be sleep but not thinking about an old frien...
7                                  http www is down 
8                       

sentiment                                            content
0       empty  i know i was bad habit earlier i started at part 
1     sadness                          n bed a headache on call 
2     sadness                    funeral ceremony gloomy friday 
3  enthusiasm                       wants hang out friends soon 
4     neutral  want trade someone who has houston tickets but...
5       worry        re pinging why t go prom bc t like friends 
6     sadness  i be sleep but not thinking about an old frien...
7       worry                                  http www is down 
8     sadness                                       love i miss 
9     sadness                    i m sorry at least it s friday

In [9]:
df = df.dropna(how='all')
for i in range(0, len(df)):
    if df.sentiment[i] == "empty":
        df = df.drop(i)
df.head(10)

sentiment                                            content
1      sadness                          n bed a headache on call 
2      sadness                    funeral ceremony gloomy friday 
3   enthusiasm                       wants hang out friends soon 
4      neutral  want trade someone who has houston tickets but...
5        worry        re pinging why t go prom bc t like friends 
6      sadness  i be sleep but not thinking about an old frien...
7        worry                                  http www is down 
8      sadness                                       love i miss 
9      sadness                    i m sorry at least it s friday 
10     neutral                                  cant fall asleep

In [10]:
sentiments = df.sentiment.unique()
print(universal_word_list[:10])
print(sentiments)

['i', 'know', 'was', 'bad', 'habit', 'earlier', 'started', 'at', 'part', 'n']
['sadness' 'enthusiasm' 'neutral' 'worry' 'surprise' 'love' 'fun' 'hate'
 'happiness' 'boredom' 'relief' 'anger']


In [11]:
key_to_word = {i:universal_word_list[i] for i in range(0, len(universal_word_list))}
word_to_key = {val:key for (key, val) in key_to_word.items()}

In [12]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= 40)
tokenizer.fit_on_texts(df['content'])
sequences = tokenizer.texts_to_sequences(df['content'])
data = pad_sequences(sequences, maxlen=50)
X = data

Using TensorFlow backend.


In [13]:
import pandas as pd
import numpy as np
flag = pd.Series(list(df['sentiment']))
Y = pd.get_dummies(flag)
Y = np.array(Y)

In [14]:
total = len(X)
ratio = 0.9
flag = int(total*ratio)
X_train = X[:flag]
X_test = X[flag:]
Y_train = Y[:flag]
Y_test = Y[flag:]

In [25]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout
from numpy import array
from keras.models import load_model

<h1><B>Bag of Words model</B></h1>

In [26]:
model = Sequential()
model.add(LSTM(100,input_shape=(50,1),return_sequences=False))
model.add(Dense(30,kernel_initializer="normal",activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(20,kernel_initializer="normal",activation="relu"))
model.add(Dense(len(sentiments),kernel_initializer="normal",activation="softmax"))
model.compile(loss="categorical_crossentropy",optimizer ="adam",metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dense_10 (Dense)             (None, 30)                3030      
_________________________________________________________________
dropout_1 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 20)                620       
_________________________________________________________________
dense_12 (Dense)             (None, 12)                252       
Total params: 44,702
Trainable params: 44,702
Non-trainable params: 0
_________________________________________________________________


In [27]:
X = X.reshape((len(X), 50, 1))
model.fit(X,Y,epochs=1,batch_size=128,validation_split=0.05);
model.save('train_model.h5')

Train on 37214 samples, validate on 1959 samples
Epoch 1/1
37214/37214 [==============================] - 27s 728us/step - loss: 2.1331 - acc: 0.2205 - val_loss: 2.1702 - val_acc: 0.1149


In [ ]:
from keras.models import load_model
model = load_model('playground_model.h5')
yhat = model.predict(X, verbose=0)
print(yhat)

<H1><B>Word Embeddings model</B></H1>

In [13]:
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [16]:
X = X.reshape(len(X), 50)
inputs = Input(shape=[50])
layer = Embedding(len(universal_word_list),500,input_length=50)(inputs)
layer = LSTM(maximum_length)(layer)
layer = Dense(256)(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(len(sentiments))(layer)
layer = Activation('sigmoid')(layer)
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])
model.fit(X,Y,batch_size=128,epochs=10,
          validation_split=0.1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 50, 500)           7813000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 31)                65968     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               8192      
_________________________________________________________________
activation_3 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 12)                3084      
__________

In [15]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15718247262839580044
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6612275691
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1189425149607929321
physical_device_desc: "device: 0, name: GeForce RTX 2070, pci bus id: 0000:08:00.0, compute capability: 7.5"
]


<H1>Reducing the classes to improve the accuracy of the model to be trained</h1>

In [70]:
two = df
categories = pd.get_dummies(df.sentiment)
two = two.drop("sentiment", axis = 1)
two = pd.concat([two, categories], axis=1, sort=False)
two = two.reset_index()

In [71]:
is_happy = []
is_sad = []
is_neutral = []

def update(happy, sad, neutral):
    is_happy.append(happy)
    is_sad.append(sad)
    is_neutral.append(neutral)

for i in two.values:
    if i[5] == 1 or i[3] == 1 or i[4] == 1 or i[7] == 1 or i[11] == 1:
        update(1, 0, 0)
    elif i[10] == 1 or i[1] == 1 or i[12] == 1 or i[6] == 1 or i[2] == 1:
        update(0, 1, 0)
    elif i[8] == 1 or i[9] == 1:
        update(0, 0, 1)
    else:
        update(0, 0, 0)

is_happy = pd.DataFrame(is_happy)
is_sad = pd.DataFrame(is_sad)
is_neutral = pd.DataFrame(is_neutral)

In [72]:
two = two.drop(["index", "anger", "boredom", "enthusiasm", "fun", "happiness", "hate", "love", "neutral", "relief", "sadness", "surprise", "worry"], axis = 1)
two = pd.concat([two, is_happy, is_sad, is_neutral], axis=1, sort=False)
two

content  0  0  0
0                              n bed a headache on call   1  0  0
1                        funeral ceremony gloomy friday   1  0  0
2                           wants hang out friends soon   1  0  0
3      want trade someone who has houston tickets but...  0  0  1
4            re pinging why t go prom bc t like friends   0  0  0
5      i be sleep but not thinking about an old frien...  1  0  0
6                                      http www is down   0  0  0
7                                           love i miss   1  0  0
8                        i m sorry at least it s friday   1  0  0
9                                      cant fall asleep   0  0  1
10                                  choked on retainers   0  0  0
11                i have beat stupid song get next rude   1  0  0
12     u watch hills in london u will realise it is w...  1  0  0
13                                             got news   0  1  0
14                    storm is here electricity is gone   1  0  0
15                                               agreed   0  0  1
16     so sleepy again it s not even late i fail once...  1  0  0
17     lady gaga tweeted about not being impressed by...  0  0  0
18     are convinced i have always wanted signals did...  1  0  0
19     oh too bad i hope it gets better i been having...  0  0  0
20     wondering why i m awake at writing a new song ...  1  0  0
21     no topic maps talks at markup conference progr...  0  0  1
22     i ate i don t know it is why do i keep telling...  0  0  0
23     so tired i think i m definitely going get an e...  1  0  0
24     on way home n having 2 deal w underage girls d...  0  0  0
25     i m sorry people are so rude isaac get some ma...  1  0  0
26     servers still down i need hit 80 before all pa...  0  0  0
27     fudge just bs d whole paper so tired i hate sc...  1  0  0
28          i hate cancer i hate it i hate it i hate it   0  0  0
29     it is so annoying starts typing on computer in...  1  0  0
...                                                  ... .. .. ..
39143  knows a site like swedish site sl se or great ...  0  0  1
39144                                    happy birthday   0  0  1
39145                        sarah u hope u remember me   1  0  0
39146                  good luck backpack quite helping   0  1  0
39147  good morning midday nation formula one in one ...  1  0  0
39148  pretty lady happy mother s day s mother future...  0  0  1
39149                      right coursework now promise   0  0  1
39150                                               say   0  1  0
39151  bloody feds lost last statement r hounding me ...  0  0  0
39152                                       awesome get   0  1  0
39153  sitting in going home a week cant wait see fam...  0  1  0
39154                                 good luck auction   0  1  0
39155  guys have ask just ask okay accept critics com...  0  0  1
39156  not really just leaving flat now on lookout lu...  0  0  1
39157                                          oh s why   0  1  0
39158  husband is golfing amp toddler i frolic am i c...  0  1  0
39159     going watch boy in striped s hope i don t cry   0  1  0
39160  gave bikes a thorough wash degrease it grease ...  0  1  0
39161  had such amazing time last night were incredible   0  1  0
39162  snoring is so annoying n it keeps me sleeping ...  0  0  1
39163            i think lesson day is not have luggage   0  0  1
39164                   can give me link diaries please   0  0  1
39165  showing french skills things good over here lo...  0  0  1
39166  yeah twitter has many uses me it s just know i...  0  0  1
39167                                         following   0  1  0
39168                                                     0  0  1
39169                        happy mothers day all love   0  0  1
39170  happy mother s day all mommies out there be wo...  0  0  1
39171  beautiful follow me peep out new hit singles w...  0  1  0
39172  bullet train tok